# Memoria TFM Eduardo Hernanz - David Sánchez
## El presente documento trata de guiar paso a paso al lector para poder comprender las decisiones tomadas tanto a nivel de 'negocio' como técnico
### El proyecto consiste en el desarrollo de un simulador que permita optimizar el tiempo empleado por un triatleta para recorrer el segmento de ciclismo de un triatlon
### Como requisito es necesario contar con un potenciómetro instalado en la bicicleta que medirá el grado de esfuerzo realizado
### Asímismo, el simulador adaptará la estrategia de potencias según los tramos caracterizados por su pendiente y viento aparente, dando una distribución de potencias que cumpla con una media previamente definida por el usuario y no sobrepasando en ningún momento una potencia máxima
#### Para conseguir este resultado, se han desarrollado las siguientes piezas de código
#### 1 - Scraping de las condiciones climatológicas de los entrenamientos usados para entrenar el modelo
#### 2 - Limpieza y creación de 'features' en los entrenamientos
#### 3 - Red Neuronal para estimar la velocidad media necesaria para recorrer un tramo en función de la potencia aplicada, el desnivel y el viento aparente
#### 4 - Simulador que prueba sobre cada tramo del recorrido objetivo diferentes potencias

In [83]:
#La Potencia viene dada por la siguiente formula

$P = \frac{\rho}{2}(V+wV)^2VCdA+(Cr+\%)mgV$

Dónde:

P es la potencia necesaria

$\rho$ es la densidad del aire

V es la velocidad

wV es velocidad el viento

Cd es el coeficiente de drag

A es el área efectiva

Cr es el coeficiente de rozamiento

$\%$ es la pendiente

m es la masa total del conjunto bicicleta - ciclista

In [148]:
import plotly as py
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import HTML
from IPython.html.widgets import interact
import math as mt

py.offline.init_notebook_mode(connected=True)

#### Suponiendo que no hay viento y se rueda en llano

La potencia total es:

$P = \frac{\rho}{2}(V)^3CdA+CrmgV$

Y se puede descomponer en la potencia necesaria para vencer al viento y para vencer el rozamiento

$Pa = \frac{\rho}{2}(V)^3CdA$

$Pr = CrmgV$


In [160]:
def show_graph(cda,por,wind):

    V = np.arange(0,100,0.5)

    ro = 1.2
    #cda = 0.3
    cr = 0.0033
    m = 90
    g = 9.81
    s = mt.sin(mt.atan(por/100))

    Pa = ro/2*cda*((V/3.6)+(wind/3.6))**2*(V/3.6)
    Pr = (cr+s)*m*g*(V/3.6)
    Pt = Pa + Pr

    trace0 = go.Scatter(
        x=V,
        y=Pa,
        mode='lines',
        name='Pot_aerodinamica'
    )

    trace1 = go.Scatter(
        x=V,
        y=Pr,
        mode='lines',
        name='Pot_rozamiento_pendinte'
    )

    trace2 = go.Scatter(
        x=V,
        y=Pt,
        mode='lines',
        name='Pot_total'
    )

    data = [trace0, trace1, trace2]

    fig = dict(data=data)

    py.offline.iplot(fig)
interact(show_graph, cda=(0.1,0.5,0.05), por=(-10,10,0.5), wind=(-15,15,1))

<function __main__.show_graph>